In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, IBMQ, Aer

In [12]:
APItoken='your APItoken'

In [ ]:
IBMQ.enable_account(APItoken)
#enable your account

In [ ]:
provider= IBMQ.get_provider()
provider.backends()
#get provider first

In [9]:
backend=provider.get_backend('ibmq_16_melbourne')

In [3]:
simulator = Aer.get_backend('qasm_simulator')
#simulator

## We design a quantum circuit to obtain Pt3

In [22]:
def Pt3(t):
    q=QuantumRegister(5)
    c=ClassicalRegister(4)
    qc=QuantumCircuit(q,c)
    qc.h(q[1])
    qc.cx(q[1],q[2])
    qc.cx(q[2],q[3])
    qc.cx(q[3],q[4])
    #produce the GHZ state
    qc.barrier()
    # If we don't add the barrier, IBM Q will automatically combine the UU^{dagger}=I . And the circuit would not perform the real processing
    qc.cx(q[3],q[4])
    qc.cx(q[2],q[3])
    qc.cx(q[1],q[2])
    qc.h(q[1])
    qc.measure(q[1],c[0])
    qc.measure(q[2],c[1])
    qc.measure(q[3],c[2])
    qc.measure(q[4],c[3])
    #assign the number of each qubits
    return qc

## By using the ancilla qubit, we can obtain the two-time correlation function by the following code (Direct-measre scenario)

In [6]:
def Pt2t3(t):
    q=QuantumRegister(14)
    c=ClassicalRegister(8)
    qc=QuantumCircuit(q,c)
    qc.h(q[1])
    qc.cx(q[1],q[2])
    qc.cx(q[2],q[3])
    qc.cx(q[3],q[4])
   #produce the GHZ state
    qc.cx(q[1],q[13])
    qc.cx(q[2],q[12])
    qc.cx(q[3],q[11])
    qc.cx(q[4],q[10])
    #q[13] q[12] q[11] q[10] are ancilla qubits
    #they are used to measure the GHZ state containing q[1] q[2] q[3] q[4] at t2 respectively
    qc.cx(q[3],q[4])
    qc.cx(q[2],q[3])
    qc.cx(q[1],q[2])
    qc.h(q[1])
    qc.measure(q[13],c[0])
    qc.measure(q[12],c[1])
    qc.measure(q[11],c[2])
    qc.measure(q[10],c[3])
    qc.measure(q[1],c[4])
    qc.measure(q[2],c[5])
    qc.measure(q[3],c[6])
    qc.measure(q[4],c[7])
    #assign the number of each qubits
    return qc

## The usefulness of t is making the circuit run many times in a loop

# Get Pt3

In [23]:
A=[]
for t in range(5):
    job=execute(Pt3(0),simulator,shots=8192)
    A.append(job)

In [24]:
for i in range(5):
    print(A[i].result().get_counts())

{'0000': 8192}
{'0000': 8192}
{'0000': 8192}
{'0000': 8192}
{'0000': 8192}


In [25]:
f={}
f[0]={'0000': 8192}
f[1]={'0000': 8192}
f[2]={'0000': 8192}
f[3]={'0000': 8192}
f[4]={'0000': 8192}

# Pick Pt3 with only outcome 0000 which is used in quantum witness


In [21]:
#Transfer the data from IBM Q become probability
import copy
def pick_t3(g,n):
    f=copy.deepcopy(g)
    for i in range(len(f)):
        for key in f[i].keys():
            f[i][key] = f[i][key]/8192
            if key == n:
                print(f[i][n])

In [26]:
pick_t3(f,'0000')

1.0
1.0
1.0
1.0
1.0


# Obtain two-time correlation function Pt2*Pt2t3

In [11]:
A=[]
for t in range(5):
    job=execute(Pt2t3(t),simulator,shots=8192)
    A.append(job)

In [13]:
for i in range(5):
    print(A[i].result().get_counts())

{'00010000': 2017, '00000000': 2115, '00011111': 2094, '00001111': 1966}
{'00010000': 2119, '00000000': 2057, '00011111': 1972, '00001111': 2044}
{'00010000': 1997, '00000000': 2064, '00011111': 2086, '00001111': 2045}
{'00010000': 2014, '00000000': 2021, '00011111': 2099, '00001111': 2058}
{'00010000': 2037, '00000000': 2026, '00011111': 2032, '00001111': 2097}


In [14]:
import copy
def picks(g,n,k):
    # g is the raw data
    # n represents the number of the entangled qubits you used
    # k is the final state you want from the raw data
    effective_f={}
    effective={}
    f=copy.deepcopy(g)
    # we don't want the data to be changed, so we make a copy of that
    for i in range(len(f)):
        for key in f[i].keys():
            f[i][key] = f[i][key]/8192
        data=f[i].keys()
        state=[]
        for j in data:
            state.append(j)
        final_state=[u[0:n] for u in state]
        effective_state=[]
        for j in range(len(state)):
            if final_state[j] == k :
                effective_state.append(state[j])
                    
        effective[i]={key:f[i][key] for key in effective_state}
        
        
            
    return effective

In [15]:
g={}
g[0]={'00010000': 2017, '00000000': 2115, '00011111': 2094, '00001111': 1966}
g[1]={'00010000': 2119, '00000000': 2057, '00011111': 1972, '00001111': 2044}
g[2]={'00010000': 1997, '00000000': 2064, '00011111': 2086, '00001111': 2045}
g[3]={'00010000': 2014, '00000000': 2021, '00011111': 2099, '00001111': 2058}
g[4]={'00010000': 2037, '00000000': 2026, '00011111': 2032, '00001111': 2097}

In [17]:
picks(g,4,'0000')
# the sum of data is Pt2*Pt2t3

{0: {'00000000': 0.2581787109375, '00001111': 0.239990234375},
 1: {'00000000': 0.2510986328125, '00001111': 0.24951171875},
 2: {'00000000': 0.251953125, '00001111': 0.2496337890625},
 3: {'00000000': 0.2467041015625, '00001111': 0.251220703125},
 4: {'00000000': 0.247314453125, '00001111': 0.2559814453125}}

# We can get the witness by |Pt3 - sum(Pt2*Pt2t3)|